In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('input.csv',encoding='iso-8859-1')
df = df[['prod_A','prod_B','estado','canal','gec','lift','confidence', 'sales_a','sales_b','support_b']]

for column in ['prod_A','prod_B','estado','canal','gec']:
    df[column] = df[column].str.upper()
    
df = df.dropna()
df

,prod_A,prod_B,estado,canal,gec,lift,confidence,sales_a,sales_b,support_b
0,COCA COLA 600 ML 24 PK,CC 355ML NR PET 12PK,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.004435,0.973156,2.874746e+07,1.011189e+07,0.968859
1,COCA COLA 600 ML 24 PK,COCA COLA 3 LT PET NR 4 PK,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.006433,0.967023,2.874746e+07,2.351121e+07,0.960842
2,COCA COLA 600 ML 24 PK,COCA COLA 2.5 LT REF-PET 8 CJ,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.003575,0.945397,2.874746e+07,3.279004e+07,0.942028
3,COCA COLA 600 ML 24 PK,CC PET 1LTNR 12 PK,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.008326,0.949475,2.874746e+07,1.115415e+07,0.941635
4,COCA COLA 600 ML 24 PK,SIDRAL 600 ML PET NR 12 G,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.007613,0.945274,2.874746e+07,3.851277e+06,0.938132
...,...,...,...,...,...,...,...,...,...,...
2639247,CIEL 600 ML PET NR 12 PK,SIDRAL 355ML LATA LEAN NR 12PK,VDM ORIENTE,OTROS,PLATA,9.000000,1.000000,1.141223e+01,7.500000e-01,0.111111
2639248,CC PET 1LTNR 12 PK,SIDRAL 355ML LATA LEAN NR 12PK,VDM ORIENTE,OTROS,PLATA,0.000000,0.000000,1.056688e+02,7.500000e-01,0.111111
2639249,CIEL 600 ML PET NR 12 PK,DV EDGE 946ML 4 PK NR NARANJA NECTAR,VDM ORIENTE,OTROS,PLATA,9.000000,1.000000,1.141223e+01,6.664180e-01,0.111111
2639250,CC PET 1LTNR 12 PK,DV EDGE 946ML 4 PK NR NARANJA NECTAR,VDM ORIENTE,OTROS,PLATA,0.000000,0.000000,1.056688e+02,6.664180e-01,0.111111


In [3]:
df_sales = df[['prod_A','sales_a']]
df_sales.columns = ['product','sales']
df_sales = df_sales.groupby(by=["product"]).sum()
df_sales['category'] = 'NA'

df_sales.loc[(df_sales['sales'] < df_sales['sales'].quantile(.25)),'category'] = 'BAJA VENTA'
df_sales.loc[(df_sales['sales'] >= df_sales['sales'].quantile(.25)) & (df_sales['sales'] < df_sales['sales'].quantile(.5)),'category'] = 'MEDIA VENTA'
df_sales.loc[(df_sales['sales'] >= df_sales['sales'].quantile(.5)) & (df_sales['sales'] < df_sales['sales'].quantile(.75)),'category'] = 'ALTA VENTA'
df_sales.loc[(df_sales['sales'] >= df_sales['sales'].quantile(.75)),'category'] = 'MUY ALTA VENTA'

df_sales = df_sales.reset_index()
df_sales


,product,sales,category
0,1C LECHE UHT ENT 1L TRP NR 12P,6.013751e+07,MEDIA VENTA
1,ADES CHOCOLATE 200 ML 4PK,2.390912e+07,MEDIA VENTA
2,ADES FRUT DNO 946 ML 12 PK,3.252484e+07,MEDIA VENTA
3,ADES FRUT MGO 200 ML 4 PK,1.021864e+07,MEDIA VENTA
4,ADES FRUT MGO 946 ML 3 PK,9.669178e+07,ALTA VENTA
...,...,...,...
572,YOGURT PLÃ¡TANO 150 GR,7.143157e+03,BAJA VENTA
573,YOGURT PLÃ¡TANO 500 GR,8.571821e+03,BAJA VENTA
574,YOLI LIMON BOT 2L NR 8PK,6.450587e+08,MUY ALTA VENTA
575,YOLI LIMON BOT 600L NR 12PK,8.554939e+08,MUY ALTA VENTA


In [4]:
df =df.merge(df_sales[['product','category']], left_on='prod_B',right_on='product', how='left')
df.columns = df.columns.str.replace('category', 'category_b')
del df['product']
df

,prod_A,prod_B,estado,canal,gec,lift,confidence,sales_a,sales_b,support_b,category_b
0,COCA COLA 600 ML 24 PK,CC 355ML NR PET 12PK,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.004435,0.973156,2.874746e+07,1.011189e+07,0.968859,MUY ALTA VENTA
1,COCA COLA 600 ML 24 PK,COCA COLA 3 LT PET NR 4 PK,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.006433,0.967023,2.874746e+07,2.351121e+07,0.960842,MUY ALTA VENTA
2,COCA COLA 600 ML 24 PK,COCA COLA 2.5 LT REF-PET 8 CJ,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.003575,0.945397,2.874746e+07,3.279004e+07,0.942028,MUY ALTA VENTA
3,COCA COLA 600 ML 24 PK,CC PET 1LTNR 12 PK,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.008326,0.949475,2.874746e+07,1.115415e+07,0.941635,MUY ALTA VENTA
4,COCA COLA 600 ML 24 PK,SIDRAL 600 ML PET NR 12 G,VDM ORIENTE,PUNTO DE VENTA TRADICIONAL,ORO,1.007613,0.945274,2.874746e+07,3.851277e+06,0.938132,MUY ALTA VENTA
...,...,...,...,...,...,...,...,...,...,...,...
2486377,CIEL 600 ML PET NR 12 PK,SIDRAL 355ML LATA LEAN NR 12PK,VDM ORIENTE,OTROS,PLATA,9.000000,1.000000,1.141223e+01,7.500000e-01,0.111111,MUY ALTA VENTA
2486378,CC PET 1LTNR 12 PK,SIDRAL 355ML LATA LEAN NR 12PK,VDM ORIENTE,OTROS,PLATA,0.000000,0.000000,1.056688e+02,7.500000e-01,0.111111,MUY ALTA VENTA
2486379,CIEL 600 ML PET NR 12 PK,DV EDGE 946ML 4 PK NR NARANJA NECTAR,VDM ORIENTE,OTROS,PLATA,9.000000,1.000000,1.141223e+01,6.664180e-01,0.111111,ALTA VENTA
2486380,CC PET 1LTNR 12 PK,DV EDGE 946ML 4 PK NR NARANJA NECTAR,VDM ORIENTE,OTROS,PLATA,0.000000,0.000000,1.056688e+02,6.664180e-01,0.111111,ALTA VENTA


In [5]:
df.to_csv('metrics.csv', index=False)
df_sales.to_csv('sales.csv', index=False)